In [1]:
cd E:\dog-breed-classifier

E:\dog-breed-classifier


In [2]:
import utils_channels_last
from utils_channels_last import Vgg16BN


Using TensorFlow backend.


In [3]:
import numpy as np
from utils_channels_last import Vgg16BN
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model

## Classifying the Dog Breed Using (Transfer Leraning)
Here we are using  pretrained VGG16BN network .Since there are less no. of images per class.There is a high chance of overfitting so we use VGG16 network with Batch Normalization.

In [4]:
batch_size=64##This parameter should Be set according to your GPU/CPU capacity . Else Be Ready FOr Exhaust Error!
num_classes = 120
image_size = 224
num_channels = 3

## IMAGE DATA  AUGMENTATION 
Here we use data Augmentation as Described in the previous post.

In [5]:
train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory('cropped/train', target_size=(224,224),
            class_mode='categorical', shuffle=True, batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory('cropped/test', target_size=(224,224),
            class_mode='categorical', shuffle=False, batch_size=batch_size)

Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [6]:
base_model = Vgg16BN(include_top=False).model
x = base_model.output
x = Flatten()(x)
x = Dropout(0.4)(x)
# let's add two fully-connected layer
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
# and a softmax layer -- let's say we have 200 classes
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

E:\dog-breed-classifier\utils_channels_last.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
E:\dog-breed-classifier\utils_channels_last.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
E:\dog-breed-classifier\utils_channels_last.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
E:\dog-breed-classifier\utils_channels_last.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
C:\Users\SANYAM\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("la.

In [8]:
model.fit_generator(train_generator, samples_per_epoch=train_generator.nb_sample, nb_epoch=10, 
                        validation_data=validation_generator, nb_val_samples=validation_generator.nb_sample)

Epoch 1/10
12000/12000 [==============================] - 520s - loss: 3.5297 - acc: 0.2190 - val_loss: 1.4586 - val_acc: 0.5924
Epoch 2/10
12000/12000 [==============================] - 520s - loss: 1.8535 - acc: 0.4918 - val_loss: 1.0614 - val_acc: 0.6801
Epoch 3/10
12000/12000 [==============================] - 520s - loss: 1.3982 - acc: 0.5973 - val_loss: 0.9268 - val_acc: 0.7228
Epoch 4/10
12000/12000 [==============================] - 520s - loss: 1.1430 - acc: 0.6581 - val_loss: 0.8699 - val_acc: 0.7353
Epoch 5/10
12000/12000 [==============================] - 520s - loss: 0.9977 - acc: 0.7038 - val_loss: 0.8191 - val_acc: 0.7471
Epoch 6/10
12000/12000 [==============================] - 520s - loss: 0.8887 - acc: 0.7325 - val_loss: 0.7997 - val_acc: 0.7506
Epoch 7/10
12000/12000 [==============================] - 520s - loss: 0.7966 - acc: 0.7564 - val_loss: 0.8051 - val_acc: 0.7542
Epoch 8/10
12000/12000 [==============================] - 521s - loss: 0.7326 - acc: 0.7716 - val

In [9]:
model.optimizer.lr = 1e6
model.fit_generator(train_generator, samples_per_epoch=train_generator.nb_sample, nb_epoch=10, 
                        validation_data=validation_generator, nb_val_samples=validation_generator.nb_sample)

Epoch 1/10
12000/12000 [==============================] - 521s - loss: 0.6018 - acc: 0.8064 - val_loss: 0.7918 - val_acc: 0.7621
Epoch 2/10
12000/12000 [==============================] - 520s - loss: 0.5409 - acc: 0.8280 - val_loss: 0.7746 - val_acc: 0.7693
Epoch 3/10
12000/12000 [==============================] - 520s - loss: 0.5194 - acc: 0.8340 - val_loss: 0.7987 - val_acc: 0.7671
Epoch 4/10
12000/12000 [==============================] - 520s - loss: 0.4842 - acc: 0.8462 - val_loss: 0.7950 - val_acc: 0.7633
Epoch 5/10
12000/12000 [==============================] - 520s - loss: 0.4513 - acc: 0.8544 - val_loss: 0.7906 - val_acc: 0.7670
Epoch 6/10
12000/12000 [==============================] - 520s - loss: 0.4190 - acc: 0.8658 - val_loss: 0.7987 - val_acc: 0.7678
Epoch 7/10
12000/12000 [==============================] - 520s - loss: 0.4149 - acc: 0.8697 - val_loss: 0.7999 - val_acc: 0.7723
Epoch 8/10
12000/12000 [==============================] - 520s - loss: 0.3958 - acc: 0.8748 - val

In [12]:
model.save_weights('vgg_model_weights.h5')